# 투표기반 앙상블

---

## Part1.분류(Classification)

### 1.분석 데이터 준비

In [1]:
import warnings
warnings.filterwarnings("ignore")
import pandas as pd

In [2]:
data1=pd.read_csv('data/breast-cancer-wisconsin.csv', encoding='utf-8')
data1.head()

,code,Clump_Thickness,Cell_Size,Cell_Shape,Marginal_Adhesion,Single_Epithelial_Cell_Size,Bare_Nuclei,Bland_Chromatin,Normal_Nucleoli,Mitoses,Class
0,1000025,5,1,1,1,2,1,3,1,1,0
1,1002945,5,4,4,5,7,10,3,2,1,0
2,1015425,3,1,1,1,2,2,3,1,1,0
3,1016277,6,8,8,1,3,4,3,7,1,0
4,1017023,4,1,1,3,2,1,3,1,1,0


In [3]:
# 특성데이터 분류
X=data1[data1.columns[1:10]]
y=data1[["Class"]]

In [4]:
# 학습, 테스트 데이터
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test=train_test_split(X, y, stratify=y, random_state=42)

In [5]:
# 정규화
from sklearn.preprocessing import MinMaxScaler
scaler=MinMaxScaler()
scaler.fit(X_train)
X_scaled_train=scaler.transform(X_train)
X_scaled_test=scaler.transform(X_test)

### 2.강한 학습기 : hard learner
- VotingClassifier

In [6]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.ensemble import VotingClassifier

In [7]:
logit_model = LogisticRegression(random_state=42)
rnf_model = RandomForestClassifier(random_state=42)
svm_model = SVC(random_state=42)

In [11]:
voting_hard = VotingClassifier(estimators=[('lr',logit_model),
                                             ('rf',rnf_model),
                                             ('svc',svm_model)])
voting_hard.fit(X_scaled_train,y_train)

VotingClassifier(estimators=[('lr',
                              LogisticRegression(C=1.0, class_weight=None,
                                                 dual=False, fit_intercept=True,
                                                 intercept_scaling=1,
                                                 l1_ratio=None, max_iter=100,
                                                 multi_class='auto',
                                                 n_jobs=None, penalty='l2',
                                                 random_state=42,
                                                 solver='lbfgs', tol=0.0001,
                                                 verbose=0, warm_start=False)),
                             ('rf',
                              RandomForestClassifier(bootstrap=True,
                                                     ccp_alpha=0.0,
                                                     class_weight=None,
                                               

In [12]:
# 정확도 확인
from sklearn.metrics import accuracy_score

for clf in (logit_model, rnf_model, svm_model, voting_hard):
    clf.fit(X_scaled_train, y_train)
    y_pred = clf.predict(X_scaled_test)
    print(clf.__class__.__name__, accuracy_score(y_test, y_pred))

LogisticRegression 0.9590643274853801
RandomForestClassifier 0.9649122807017544
SVC 0.9649122807017544
VotingClassifier 0.9649122807017544


In [15]:
# 로지스틱 분류기 오차행렬
from sklearn.metrics import confusion_matrix
log_pred_train = logit_model.predict(X_scaled_train)
log_confusion_train = confusion_matrix(y_train, log_pred_train)
print('로지스틱 분류기 훈련데이터 오차행렬:\n', log_confusion_train)

log_pred_test = logit_model.predict(X_scaled_test)
log_confusion_test = confusion_matrix(y_test, log_pred_test)
print('로지스틱 분류기 테스트데이터 오차행렬:\n', log_confusion_test)

로지스틱 분류기 훈련데이터 오차행렬:
 [[328   5]
 [  9 170]]
로지스틱 분류기 테스트데이터 오차행렬:
 [[106   5]
 [  2  58]]


In [16]:
# 서포트벡터머신 분류기
svm_pred_train=svm_model.predict(X_scaled_train)
svm_confusion_train=confusion_matrix(y_train, svm_pred_train)
print("서포트벡터머신 분류기 훈련데이터 오차행렬:\n", svm_confusion_train)

svm_pred_test=svm_model.predict(X_scaled_test)
svm_confusion_test=confusion_matrix(y_test, svm_pred_test)
print("서포트벡터머신 분류기 훈련데이터 오차행렬:\n", svm_confusion_test)

서포트벡터머신 분류기 훈련데이터 오차행렬:
 [[329   4]
 [  4 175]]
서포트벡터머신 분류기 훈련데이터 오차행렬:
 [[106   5]
 [  1  59]]


In [17]:
# 랜덤포레스트 분류기
rnd_pred_train=rnf_model.predict(X_scaled_train)
rnd_confusion_train=confusion_matrix(y_train, rnd_pred_train)
print("랜덤포레스트 분류기 훈련데이터 오차행렬:\n", rnd_confusion_train)

rnd_pred_test=rnf_model.predict(X_scaled_test)
rnd_confusion_test=confusion_matrix(y_test, rnd_pred_test)
print("랜덤포레스트 분류기 테스트데이터 오차행렬:\n", rnd_confusion_test)

랜덤포레스트 분류기 훈련데이터 오차행렬:
 [[333   0]
 [  0 179]]
랜덤포레스트 분류기 테스트데이터 오차행렬:
 [[106   5]
 [  1  59]]


In [18]:
# 투표분류기 분류기
voting_pred_train=voting_hard.predict(X_scaled_train)
voting_confusion_train=confusion_matrix(y_train, voting_pred_train)
print("투표분류기 분류기 훈련데이터 오차행렬:\n", voting_confusion_train)

voting_pred_test=voting_hard.predict(X_scaled_test)
voting_confusion_test=confusion_matrix(y_test, voting_pred_test)
print("투표분류기 분류기 훈련데이터 오차행렬:\n", voting_confusion_test)

투표분류기 분류기 훈련데이터 오차행렬:
 [[329   4]
 [  4 175]]
투표분류기 분류기 훈련데이터 오차행렬:
 [[106   5]
 [  1  59]]


### 3.약한 학습기 : soft learner

In [19]:
logit_model = LogisticRegression(random_state=42)
rnf_model = RandomForestClassifier(random_state=42)
svm_model = SVC(probability=True, random_state=42)

# voting='soft'
voting_soft = VotingClassifier(
    estimators=[('lr', logit_model), ('rf', rnf_model), ('svc', svm_model)], voting='soft')
voting_soft.fit(X_scaled_train, y_train)

VotingClassifier(estimators=[('lr',
                              LogisticRegression(C=1.0, class_weight=None,
                                                 dual=False, fit_intercept=True,
                                                 intercept_scaling=1,
                                                 l1_ratio=None, max_iter=100,
                                                 multi_class='auto',
                                                 n_jobs=None, penalty='l2',
                                                 random_state=42,
                                                 solver='lbfgs', tol=0.0001,
                                                 verbose=0, warm_start=False)),
                             ('rf',
                              RandomForestClassifier(bootstrap=True,
                                                     ccp_alpha=0.0,
                                                     class_weight=None,
                                               

In [20]:
from sklearn.metrics import accuracy_score

for clf in (logit_model, rnf_model, svm_model, voting_soft):
    clf.fit(X_scaled_train, y_train)
    y_pred = clf.predict(X_scaled_test)
    print(clf.__class__.__name__, accuracy_score(y_test, y_pred))

LogisticRegression 0.9590643274853801
RandomForestClassifier 0.9649122807017544
SVC 0.9649122807017544
VotingClassifier 0.9649122807017544


In [21]:
voting_pred_train=voting_soft.predict(X_scaled_train)
voting_confusion_train=confusion_matrix(y_train, voting_pred_train)
print("투표분류기 분류기 훈련데이터 오차행렬:\n", voting_confusion_train)

voting_pred_test=voting_soft.predict(X_scaled_test)
voting_confusion_test=confusion_matrix(y_test, voting_pred_test)
print("투표분류기 분류기 훈련데이터 오차행렬:\n", voting_confusion_test)

투표분류기 분류기 훈련데이터 오차행렬:
 [[330   3]
 [  3 176]]
투표분류기 분류기 훈련데이터 오차행렬:
 [[106   5]
 [  1  59]]


---

## Part2.회귀(Regression)

### 1.분석 데이터 준비

In [23]:
data2=pd.read_csv('data/house_price.csv', encoding='utf-8')
data2.head()

,housing_age,income,bedrooms,households,rooms,house_value
0,23,6.7770,0.141112,2.442244,8.103960,500000
1,49,6.0199,0.160984,2.726688,5.752412,500000
2,35,5.1155,0.249061,1.902676,3.888078,500000
3,32,4.7109,0.231383,1.913669,4.508393,500000
4,21,4.5625,0.255583,3.092664,4.667954,500000


In [24]:
# 특성데이터 분류
X=data2[data2.columns[1:5]]
y=data2[["house_value"]]

In [25]:
# 학습, 테스트 데이터 분류
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test=train_test_split(X, y, random_state=42)

In [26]:
# 정규화
from sklearn.preprocessing import MinMaxScaler
scaler=MinMaxScaler()
scaler.fit(X_train)
X_scaled_train=scaler.transform(X_train)
X_scaled_test=scaler.transform(X_test)

### 2. 모델적용

In [27]:
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import VotingRegressor

In [28]:
linear_model= LinearRegression()
rnf_model = RandomForestRegressor(random_state=42)

voting_regressor = VotingRegressor(estimators=[('lr', linear_model), ('rf', rnf_model)])
voting_regressor.fit(X_scaled_train, y_train)

VotingRegressor(estimators=[('lr',
                             LinearRegression(copy_X=True, fit_intercept=True,
                                              n_jobs=None, normalize=False)),
                            ('rf',
                             RandomForestRegressor(bootstrap=True,
                                                   ccp_alpha=0.0,
                                                   criterion='mse',
                                                   max_depth=None,
                                                   max_features='auto',
                                                   max_leaf_nodes=None,
                                                   max_samples=None,
                                                   min_impurity_decrease=0.0,
                                                   min_impurity_split=None,
                                                   min_samples_leaf=1,
                                                   min_samples_spli

In [29]:
pred_train=voting_regressor.predict(X_scaled_train)
voting_regressor.score(X_scaled_train, y_train)

0.7962532705428835

In [30]:
pred_test=voting_regressor.predict(X_scaled_test)
voting_regressor.score(X_scaled_test, y_test)

0.5936371957936409

In [31]:
# RMSE (Root Mean Squared Error)
import numpy as np
from sklearn.metrics import mean_squared_error 
MSE_train = mean_squared_error(y_train, pred_train)
MSE_test = mean_squared_error(y_test, pred_test)
print("훈련   데이터 RMSE:", np.sqrt(MSE_train))
print("테스트 데이터 RMSE:", np.sqrt(MSE_test))

훈련   데이터 RMSE: 43082.050654857834
테스트 데이터 RMSE: 60942.38524353489
